# Diesel Engine - Power Study

This is a study that will use properties in the a Capella model to perform a study on the number of solar panels required to satisfy various a few usage scenarios.

Access the System Model that is in GitHUB folder with a python module "caplellambse" that can read and modify the SMW/Capella model content.

In [1]:
import capellambse

import ipywidgets as widgets
from IPython.core.display import HTML
import numpy as np
import pandas as pd 
from IPython.display import Markdown


resources = {
    "Import-Test": "Import-Test/Import Test",
}
path_to_model = "../Import Test.aird"
model = capellambse.MelodyModel(path_to_model, resources=resources)


import capellambse_helper #local python module in same directory as notebook needs to be included after model object is created since it patches/extends capabilitits of the model
#print(model.la.root_component)
#help(model.la.root_component)
#print()
#print( model.search().by_uuid("834a4589-56b3-44f1-8836-fcf41855308b"))



Display a diagram from the capella module with parameters being accessed. Note that this diagram is not being pulled dynamically, due to the desire to see the parameters on the diagram. It is pulling a snapshot of a .jpg diagram.

In [2]:
#from IPython import display as diag_display
#diag_display.Image("../Images/[LAB] Trail Power (Charge) Properties and Constraints.jpg")


## Display all the parameters and respective components/functions in the logical architecture
The following is a report of all the parameters avialable to be used. 

In [3]:
import io
from capellambse import decl
la_model = model.la 
import yaml
from capellambse.metamodel.cs import Part
import uuid
capellambse_helper.display_function_property_values(la_model)

capellambse_helper.display_component_property_values(la_model)

#help(la_model.root_component)

# Example system components you want to create
components = [
    {"name": "Engine Block", "function": "Foundation of the engine, housing cylinders for combustion"},
    {"name": "Cylinders", "function": "Chambers where air is compressed and diesel fuel is injected"},
    {"name": "Pistons", "function": "Convert energy from combustion into mechanical energy"},
    {"name": "Crankshaft", "function": "Converts linear motion of pistons into rotational motion"},
    {"name": "Connecting Rods", "function": "Connect pistons to crankshaft, transmitting motion"},
    {"name": "Cylinder Head", "function": "Seals the top of the engine block, forming combustion chamber"},
    {"name": "Valves", "function": "Regulate the flow of air into cylinders and exhaust gases out"},
    {"name": "Camshaft", "function": "Controls the opening and closing of intake and exhaust valves"},
    {"name": "Timing Chain/Belt", "function": "Synchronizes the rotation of crankshaft and camshaft"},
    {"name": "Fuel Injectors", "function": "Delivers diesel fuel into the combustion chamber"},
    {"name": "Turbocharger", "function": "Increases air entering the engine, improving power and efficiency"},
    {"name": "Intercooler", "function": "Cools compressed air from turbocharger before entering engine"},
    {"name": "Glow Plugs", "function": "Preheats combustion chamber to assist in cold starting"},
    {"name": "Fuel Pump", "function": "Pumps diesel fuel from tank to injectors at high pressure"},
    {"name": "Exhaust Manifold", "function": "Collects exhaust gases and directs them to the exhaust system"},
    {"name": "Intake Manifold", "function": "Distributes air to the engine's cylinders"},
    {"name": "Oil Pump", "function": "Circulates oil throughout the engine for lubrication"},
    {"name": "Water Pump", "function": "Circulates coolant to regulate engine temperature"},
    {"name": "Radiator", "function": "Cools the engine by dissipating heat from the coolant"},
    {"name": "Flywheel", "function": "Stores rotational energy and smooths power pulses"},
    {"name": "Air Filter", "function": "Filters air before it enters the engine"},
    {"name": "Engine Control Unit", "function": "Controls fuel injection timing, air intake, and turbocharging"},
    {"name": "EGR System", "function": "Reduces emissions by recirculating exhaust gases"},
    {"name": "Particulate Filter", "function": "Captures soot and particulate matter from exhaust gases"}
]

# Create a function to add components to the Capella model
def create_logical_component(model,root_component,component):
    print(component["name"])
    print(root_component)
    # Define a simple constructor for the !uuid tag
    def uuid_constructor(loader, node):
        return loader.construct_scalar(node)
    
    # Register the custom constructor for !uuid
    yaml.add_constructor("!uuid", uuid_constructor, Loader=yaml.SafeLoader)
    model_update = f"""
- parent: !uuid {root_component.uuid}
  extend:
    components:
      - name: {component["name"]}
        """
        # the below line applies the model_update to the model
    print(model_update)
    try:
        # Validate YAML
        yaml.safe_load(model_update)
        print("YAML validated successfully!")
    
        try:
            # Apply the YAML to the model
            decl.apply(model, io.StringIO(model_update))
            print("Model update applied successfully.")
        except Exception as e:
             raise RuntimeError(f"YAML Application Error: {e}")
    except yaml.YAMLError as e:
        raise RuntimeError(f"YAML validation error: {e}\nProblematic YAML:\n{model_update}")

    return model.search("LogicalComponent").by_name(component["name"])

def create_logical_component_part(model, root_component, comp_for_type):
    import io
    print(comp_for_type.name)
    print(root_component)
    
    # Correctly formatted YAML
    def uuid_constructor(loader, node):
        return loader.construct_scalar(node)
    
    # Register the custom constructor for !uuid
    yaml.add_constructor("!uuid", uuid_constructor, Loader=yaml.SafeLoader)
    model_update = f"""
- parent: !uuid {root_component.uuid}
  extend:
    parts:
      - name: {comp_for_type.name}
        type: {comp_for_type.uuid}
    """
    
    # Print YAML for debugging
    print("Generated YAML for model update:\n", model_update)
    try:
        # Validate YAML
        yaml.safe_load(model_update)
        print("YAML validated successfully!")
    
        try:
            # Apply the YAML to the model
            decl.apply(model, io.StringIO(model_update))
            print("Model update applied successfully.")
        except Exception as e:
            raise RuntimeError(f"YAML Application Error: {e}")
    except yaml.YAMLError as e:
        raise RuntimeError(f"YAML validation error: {e}\nProblematic YAML:\n{model_update}")

   
def create_logical_function(model,root_component,function):
    print(function["name"])
    def uuid_constructor(loader, node):
        return loader.construct_scalar(node)
    
    # Register the custom constructor for !uuid
    yaml.add_constructor("!uuid", uuid_constructor, Loader=yaml.SafeLoader)
    model_update = f"""
- parent: !uuid {root_component.uuid}
  extend:
    functions:
        - name: {function["function"]}
            """
        # the below line applies the model_update to the model
    try:
        # Validate YAML
        yaml.safe_load(model_update)
        print("YAML validated successfully!")

    
        try:
            # Apply the YAML to the model
            decl.apply(model, io.StringIO(model_update))
            print("Model update applied successfully.")
        except Exception as e:
            raise RuntimeError(f"YAML Application Error: {e}")
    except yaml.YAMLError as e:
        raise RuntimeError(f"YAML validation error: {e}\nProblematic YAML:\n{model_update}")


def add_part_to_component(model, parent_component, comp):
    """Manually add a part to a logical component."""
    new_uuid = str(uuid.uuid4())
    print(parent_component)
    print(parent_component._element)
    print(comp)
    help(Part)
    new_part = Part(model = model,  parent=parent_component._element, type = comp, name = comp.name, uuid= new_uuid )  
    try:
        # Manually append the new part using the parent component's XML

        
        #part = model.create("Part", name=comp.name, type=comp)
        parent_component.parts.append(part)  # If `parts` is a list-like object
        print(f"Part '{comp.name}' added successfully to '{parent_component.name}'.")
    except AttributeError as e:
        print(f"Attribute error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Add components to the Capella model
for comp in components:
    new_comp = create_logical_component(model,la_model.root_component,comp)
    add_part_to_component(model, la_model.root_component, new_comp )
    #create_logical_component_part(model,la_model.root_component,new_comp)
for comp in components:
    create_logical_function(model,la_model.root_function,comp)
#for component in model.la.root_component.components:
#    print(component.parts[0])
#    #capellambse.decl.dump(component)
#model.save()
print("Components successfully added to the Capella model.")




,Logical Function,Property Value Group Name,Propery Name,Property Value


,Logical Component,Property Value Group Name,Property Name,Property Value


Engine Block
<LogicalComponent 'Logical System' (896ef2aa-001d-4219-96e8-4a018cae7cfe)>
.allocated_functions = []
.applied_property_value_groups = []
.applied_property_values = []
.components = []
.constraints = []
.context_diagram = <Diagram 'Context of Logical System'>
.description = Markup('')
.diagrams = []
.exchanges = []
.filtering_criteria = []
.is_abstract = False
.is_actor = False
.is_human = False
.layer = <LogicalArchitecture 'Logical Architecture' (76a4d7e0-803b-40cb-bf5f-91a559ceaae4)>
.name = 'Logical System'
.owned_features = []
.owner = <LogicalComponentPkg 'Structure' (2d9ad464-b546-40eb-929f-f342751868ed)>
.parent = <LogicalComponentPkg 'Structure' (2d9ad464-b546-40eb-929f-f342751868ed)>
.parts = ... # backreference to Part - omitted: can be slow to compute
.physical_links = []
.physical_paths = []
.physical_ports = []
.ports = []
.progress_status = 'NOT_SET'
.property_value_groups = []
.property_value_packages = []
.property_values = []
.pvmt = <Property Value Manage

TypeError: ModelElement.__init__() missing 2 required positional arguments: 'model' and 'parent'